# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
回忆一下在 :numref:`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。

In [1]:
import torch
import torch.nn.functional as F
from torch import nn


print(torch.__version__)

1.13.1+cu117


In [2]:
class CenteredLayer(nn.Module):
    
    def __init__(self):
        super().__init__()

    def forward(self, X):
        y1 = X - X.mean()
        return y1

让我们向该层提供一些数据，验证它是否能按预期工作。


In [19]:
layer = CenteredLayer()

X = torch.FloatTensor([1, 2, 3, 4, 5])
print(X)

print('计算结果: ', layer(X))

tensor([1., 2., 3., 4., 5.])
计算结果:  tensor([-2., -1.,  0.,  1.,  2.])


现在，我们可以[**将层作为组件合并到更复杂的模型中**]。


In [5]:
net = nn.Sequential(
    nn.Linear(8, 128), 
    CenteredLayer()
)

In [6]:
# 这里使用自定义, 初始化初始化网络
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=1)
        nn.init.zeros_(m.bias)


net.apply(init_normal)

Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): CenteredLayer()
)

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [7]:
X2 = torch.rand(4, 8)

X2

tensor([[0.7861, 0.8148, 0.5295, 0.8833, 0.3692, 0.2868, 0.1620, 0.3579],
        [0.2347, 0.1419, 0.9300, 0.4804, 0.3373, 0.5486, 0.3061, 0.2491],
        [0.8915, 0.8216, 0.8106, 0.1906, 0.4331, 0.9964, 0.6802, 0.1719],
        [0.3417, 0.9091, 0.5275, 0.7543, 0.5729, 0.5596, 0.9659, 0.3932]])

In [8]:
Y = net(X2)
print(Y.shape)

Y.mean()

torch.Size([4, 128])


tensor(-6.7521e-09, grad_fn=<MeanBackward0>)

## [**带参数的层**]

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。


In [20]:
class MyLinear(nn.Module):
    
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        y_hat = F.relu(linear)
        return y_hat


"""
具体来说，这个神经网络结构包含一个自定义的线性层MyLinear，该层接受输入X，并输出通过矩阵乘法和偏置加法计算得到的结果。

在这个代码中，为了初始化权重和偏差，我们使用了nn.Parameter函数来创建权重和偏差参数。

这表示这些参数将会被优化器更新以最小化损失函数。在forward方法中，我们首先使用torch.matmul函数计算输入X和权重之间的点积，
然后添加偏置项。接着我们使用激活函数ReLU将线性变换的结果进行非线性变换，从而得到最终的输出y_hat。
"""

接下来，我们实例化`MyLinear`类并访问其模型参数。


In [10]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.3959, -1.4989, -0.2543],
        [ 0.1180, -0.7197, -0.2281],
        [-0.4724, -0.2381, -1.3180],
        [ 0.2456,  0.2017, -0.8422],
        [-1.1730,  0.6457, -1.1515]], requires_grad=True)

In [11]:
linear.state_dict()

OrderedDict([('weight',
              tensor([[ 0.3959, -1.4989, -0.2543],
                      [ 0.1180, -0.7197, -0.2281],
                      [-0.4724, -0.2381, -1.3180],
                      [ 0.2456,  0.2017, -0.8422],
                      [-1.1730,  0.6457, -1.1515]])),
             ('bias', tensor([-0.4140, -1.2611, -0.6674]))])

我们可以[**使用自定义层直接执行前向传播计算**]。


In [12]:
X3 = torch.rand(2, 5)

X3

tensor([[0.1488, 0.3675, 0.2915, 0.1620, 0.4365],
        [0.9768, 0.2524, 0.7773, 0.3810, 0.4306]])

In [13]:

# 
linear(X3)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [14]:
net = nn.Sequential(
    MyLinear(64, 8), 
    MyLinear(8, 1)
)

In [16]:
X4 = torch.rand(size=(10, 64))

# 计算
y_hat = net(X4)
y_hat

tensor([[ 4.5336],
        [ 6.6808],
        [ 0.0000],
        [ 3.2334],
        [12.7250],
        [11.2090],
        [ 1.4799],
        [ 7.1822],
        [ 5.7899],
        [ 1.1124]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。